In [1]:
import pandas as pd


In [2]:
data = pd.read_csv('WMT_Grocery_202209.csv')

/var/folders/9c/67cc_3jj62d2ds4cnt_766t80000gn/T/ipykernel_7786/3266408272.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('WMT_Grocery_202209.csv')


In [3]:
data = data[["index", "CATEGORY", "PRODUCT_NAME", "PRICE_CURRENT", "PRODUCT_SIZE"]][:1000]

In [4]:
data

,index,CATEGORY,PRODUCT_NAME,PRICE_CURRENT,PRODUCT_SIZE
0,0,"Hummus, Dips, & Salsa","Marketside Roasted Red Pepper Hummus, 10 Oz",2.67,10
1,1,"Hummus, Dips, & Salsa","Marketside Roasted Garlic Hummus, 10 Oz",2.67,10
2,2,"Hummus, Dips, & Salsa","Marketside Classic Hummus, 10 Oz",2.67,10
3,3,"Hummus, Dips, & Salsa","Marketside Everything Hummus, 10 oz",2.67,10
4,4,"Hummus, Dips, & Salsa","Price's Jalapeno Dip, 12 Oz.",3.12,12
...,...,...,...,...,...
995,995,Healthy Snacks,"Great Value Pine Nuts, 4 oz",6.92,4
996,996,Healthy Snacks,"Wonderful Pistachios Roasted & Salted, 1.5 Oz,...",6.94,1.5
997,997,Healthy Snacks,Power Up Mega Omega Trail Mix from Gourmet Nut...,5.28,14
998,998,Healthy Snacks,"Blue Diamond Almonds, Whole Natural Raw Almond...",6.98,14


In [5]:
data = data.rename(columns={"index": "id", "CATEGORY": "jenis_barang_id", "PRODUCT_NAME": "nama", "PRICE_CURRENT": "harga", "PRODUCT_SIZE": "jumlah_stok"})

In [6]:
category = pd.DataFrame(set(data["jenis_barang_id"]))

In [7]:
category = category.reset_index().rename(columns={0: "kategori", "index":"id"})

In [8]:
category

,id,kategori
0,0,Pudding & Gelatin
1,1,Healthy Snacks
2,2,"Hummus, Dips, & Salsa"
3,3,Better for you
4,4,Game Time Faves
5,5,Oils & Shortening
6,6,Energy Drinks
7,7,Fresh Soups & Salads
8,8,Sparkling Water


In [9]:
def addOneId(row):
    return row["id"] + 1

category["id"] = category.apply(addOneId, axis=1)

In [10]:
category.to_csv("kategori.csv", index=False)

In [11]:
data

,id,jenis_barang_id,nama,harga,jumlah_stok
0,0,"Hummus, Dips, & Salsa","Marketside Roasted Red Pepper Hummus, 10 Oz",2.67,10
1,1,"Hummus, Dips, & Salsa","Marketside Roasted Garlic Hummus, 10 Oz",2.67,10
2,2,"Hummus, Dips, & Salsa","Marketside Classic Hummus, 10 Oz",2.67,10
3,3,"Hummus, Dips, & Salsa","Marketside Everything Hummus, 10 oz",2.67,10
4,4,"Hummus, Dips, & Salsa","Price's Jalapeno Dip, 12 Oz.",3.12,12
...,...,...,...,...,...
995,995,Healthy Snacks,"Great Value Pine Nuts, 4 oz",6.92,4
996,996,Healthy Snacks,"Wonderful Pistachios Roasted & Salted, 1.5 Oz,...",6.94,1.5
997,997,Healthy Snacks,Power Up Mega Omega Trail Mix from Gourmet Nut...,5.28,14
998,998,Healthy Snacks,"Blue Diamond Almonds, Whole Natural Raw Almond...",6.98,14


In [12]:
def tansformData(row):
    row['harga'] = row['harga'] * 10000
    row['id'] = row['id'] + 1

    return row

data = data.apply(tansformData, axis=1)

In [13]:
data["harga"] = data["harga"].astype(int)

In [46]:
category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        114 non-null    int64 
 1   kategori  114 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.9+ KB


In [14]:
category = category.to_dict("records")
def changeCategory(row):

    for cat in category:
        if row["jenis_barang_id"] == cat["kategori"]:
            row["jenis_barang_id"] = str(cat["id"])
            break
    
    return row

data = data.apply(changeCategory, axis=1)

In [18]:
data.to_csv("barang.csv", index=False)

##### Insert to SQL

In [21]:
kategori = pd.read_csv("kategori.csv")

In [16]:
kategori

,id,kategori
0,1,Pudding & Gelatin
1,2,Healthy Snacks
2,3,"Hummus, Dips, & Salsa"
3,4,Better for you
4,5,Game Time Faves
5,6,Oils & Shortening
6,7,Energy Drinks
7,8,Fresh Soups & Salads
8,9,Sparkling Water


In [22]:
import requests

url = "http://localhost:3000/jenis-barang"

def reqInsert(row):
    myobj = {
        "id": row["id"],
        "kategori": row["kategori"]
    }

    requests.post(url, json = myobj)

kategori.apply(reqInsert, axis=1)

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
dtype: object

In [19]:
produk = pd.read_csv("barang.csv")

produk.dropna(inplace=True)

produk.to_csv("barang.csv", index=False)

In [20]:
import requests
import math
from tqdm.auto import tqdm

tqdm.pandas()
url = "http://localhost:3000/barang"

def reqInsert(row):
    try:
        float(row["jumlah_stok"])
    except:
        row["jumlah_stok"] = "0"

    myobj = {
        "id": row["id"],
        "nama": row["nama"],
        "harga": row["harga"],
        "jumlah_stok": int(math.floor(float(row["jumlah_stok"]))),
        "jenis_barang_id": row["jenis_barang_id"]
    }

    requests.post(url, json = myobj)

produk.progress_apply(reqInsert, axis=1)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 951/951 [00:02<00:00, 451.17it/s]


0      None
1      None
2      None
3      None
4      None
       ... 
995    None
996    None
997    None
998    None
999    None
Length: 951, dtype: object